# Language Modelling in Hangman

Author: Muhammad Atif

Python version used: 3.6

## General info

<b>Overview</b>: Development of an 'artificial intelligence' player for the classic Hangman word guessing game by implementing several different automatic strategies based on character level language models, ranging from unigram approaches to higher order n-gram models. Objective is to create an automatic player which makes the fewest mistakes.

## The Hangman Game

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game, and a method allowing interactive play. 

In [1]:
# allowing better python 2 & python 3 compatibility 
from __future__ import print_function 

def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

def human(mask, guessed, **kwargs):
    """
    simple function for manual play
    """
    print('Enter your guess:')
    try:
        return raw_input().lower().strip() # python 3
    except NameError:
        return input().lower().strip() # python 2

Game can be played interactively using the following command:

In [2]:
hangman('whatever', human, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ length 8
You have 8 attempts remaining.
Enter your guess:
w
Guess is w
Good guess: w _ _ _ _ _ _ _
You have 8 attempts remaining.
Enter your guess:
h
Guess is h
Good guess: w h _ _ _ _ _ _
You have 8 attempts remaining.
Enter your guess:
a
Guess is a
Good guess: w h a _ _ _ _ _
You have 8 attempts remaining.
Enter your guess:
t
Guess is t
Good guess: w h a t _ _ _ _
You have 8 attempts remaining.
Enter your guess:
e
Guess is e
Good guess: w h a t e _ e _
You have 8 attempts remaining.
Enter your guess:
v
Guess is v
Good guess: w h a t e v e _
You have 8 attempts remaining.
Enter your guess:
e
Guess is e
Already guessed this before.
You have 7 attempts remaining.
Enter your guess:
r
Guess is r
Good guess: w h a t e v e r
Congratulations, you won.


1

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the method. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

First task is to compute the unique word types occurring in the *Brown* corpus, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set.

Feel free to test Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

In [3]:
import nltk
import numpy as np
from nltk.corpus import brown   

# unique word types of lower-cased, alphabetic characters from brown corpus
wordTypes = set()
for words in brown.words():
    word = words.lower()
    if word.isalpha():
        wordTypes.add(word)       
uniqueWordTypes = list(wordTypes)

# randomly shuffle word types and split in train and test sets
np.random.shuffle(uniqueWordTypes)
testSet = uniqueWordTypes[0:1000]
trainSet = uniqueWordTypes[1000:]

# print size of train and test sets
print("Number of word types in Test Set: ", len(testSet))
print("Number of word types in Train Set: ", len(trainSet))

Number of word types in Test Set:  1000
Number of word types in Train Set:  39234


To set a baseline, our first *AI* attempt will be a trivial random method. For this we will implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Our method will randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches will also exclude previous guesses).

To measure the performance of this (and later) techiques, we will implement a method that measures the average number of mistakes made by this technique over all the words in the `test_set`. Print the average number of mistakes for the random AI, which will become a baseline for the following steps.

In [4]:
# randomly return an alphabet which has not already been guessed
def randomGuesser(mask, guessed, **kwargs):
    alphabets = set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
             'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
    alphabets = alphabets - guessed 
    return np.random.choice(list(alphabets))

# return average number of mistakes made by a given model in solving hangman across entire list of words
def hangmanPerformance(listOfWords, method, **kwargs):
    totalNoOfMistakes = 0
    for word in listOfWords:
        totalNoOfMistakes += hangman(word, method, 26, False, **kwargs)  
    return(totalNoOfMistakes / len(listOfWords))

print('Average number of mistakes for random guesser: ', hangmanPerformance(testSet, randomGuesser))  

Average number of mistakes for random guesser:  16.709


As our first real AI, we will train a *unigram* model over the training set. This will require to find the frequencies of characters over all training words. Using this model, we will write a guess function that returns the character with the highest probability, after aggregating (summing) the probability of each blank character in the secret word. Print the average number of mistakes the unigram method makes over the test set. We will exclude already guessed characters, and use the same evaluation method as above, so the results are comparable.

In [5]:
import operator

# build alphabet frequency dictionary
totalCharacters = 0
charFreqDict = {}
for words in trainSet:
    for character in words:
        charFreqDict[character] = charFreqDict.get(character, 0) + 1 
        totalCharacters += 1              

# convert character frequency into probability                 
for alphabet in charFreqDict:
    charFreqDict[alphabet] = charFreqDict[alphabet] / totalCharacters

# store ordered list of alphabets based on frequency in descending order
charFreqDict = sorted(charFreqDict.items(), key=operator.itemgetter(1), reverse=True) 
orderedCharsList = []
for characters in charFreqDict:
    orderedCharsList.append(characters[0])

# return an alphabet with highest probability across all alphabets (appearing in words) in train set
def unigramGuesser(mask, guessed, **kwargs):
    remCharsList = [chars for chars in orderedCharsList if chars not in guessed]
    return remCharsList[0]

print('Average number of mistakes for unigram guesser: ', hangmanPerformance(testSet, unigramGuesser))  

Average number of mistakes for unigram guesser:  10.211


The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. We will now incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having *different* unigram models for each length of word. We will need to be a little careful at test time, to be robust to the (unlikely) situation that we might encounter a word length that we didn't see in training. Let's create another AI guessing function using this new model, and print its test performance.   

In [6]:

import math
from collections import Counter

# get total number of different length words
lenOfWords=[]
for words in trainSet:
    lenOfWords.append(len(words))
wordLengthCounts = Counter(lenOfWords)

# get total count of alphabets for given length of word
charFreqDictByLength = {}
for words in trainSet:
    for character in words:
        charFreqDictByLength[character, len(words)] = charFreqDictByLength.get((character, len(words)),0) + 1

# convert alphabet frequencies per word length into probabilities, as follows:
# P(alphabet | wordLength) = (number of times an alphabet appeared in a word length) / (number of total alphabets across all words in that word length)
for a in charFreqDictByLength:
    charFreqDictByLength[a[0], a[1]] = charFreqDictByLength.get((a[0], a[1]), 0) / (a[1] * wordLengthCounts[a[1]])

# return probability given an alphabet and word length
# if alphabet, wordlength pair is not found check recursively in wordlength-1 
# until a pair is found or wordlength becomes 0
def getCharProbGivenLength(char,length):
    if length == 0:
        return 0.0
    else:
        if (char, length) in charFreqDictByLength:
            return charFreqDictByLength[char, length]
        else:
            return getCharProbGivenLength(char, length-1)

# return an alphabet with highest probability across all alphabets appearing in words of the same length as the secret word in train set
def lengthCondUnigramGuesser(mask, guessed):
    alphabets = set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
             'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])    
    alphabets = alphabets - guessed 
    maskLength = len(mask)
    maxFreq = (-1,'_')
    for char in alphabets:
       if getCharProbGivenLength(char,maskLength) > maxFreq[0]:
        maxFreq = (getCharProbGivenLength(char,maskLength),char)
    return maxFreq[1]

print('Average number of mistakes for length conditioned unigram guesser: ', hangmanPerformance(testSet, lengthCondUnigramGuesser))  

Average number of mistakes for length conditioned unigram guesser:  10.213


Now for the main challenge, using a *ngram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

We will now develop a *ngram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with sentinel symbols.) We will use linear interpolation to smooth between the higher order and lower order models, and will have to decide how to weight each component. 

The guessing AI algorithm should apply the language model to each blank position in the secret word by using as much of the left context as is known. E.g., in `_ e c _ e _ _` we know the full left context for the first blank (context=start of word), we have a context of two characters for the second blank (context=ec), one character for the second last blank (context=e), and no known context for the last one. If we were using a *n=3* order model, we would be able to apply it to the first and second blanks, but would only be able to use the bigram or unigram distributions for the subsequent blanks. As with the unigram model, you should sum over the probability distributions for each blank to find the expected count for each character type, then select the  character with the highest expected count.

We will implement the ngram method for *n=3,4* and *5* and evaluate each of these three models over the test set.

In [7]:

from collections import defaultdict
from collections import Counter

def convertWord(word):
    return ["<s>","<s>","<s>","<s>"] + [alphabet for alphabet in word] + ["</s>"]

def getNgramProb(dataset):
    # initialize variables
    unigramProb = Counter()
    bigramProb = defaultdict(Counter)
    trigramProb = defaultdict(lambda: defaultdict(Counter))
    quadgramProb = defaultdict(lambda: defaultdict(lambda: defaultdict(Counter)))
    pentgramProb = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(Counter))))

    # collect unigram counts
    for word in dataset:
        word = convertWord(word)
        for alphabet in word:
            unigramProb[alphabet] += 1
        unigramProb["</s>"] += 1
    
    # using alphabet frequency, compute probability of a given alphabet across all alphabets (appearing in words) in train set
    for alphabet in unigramProb:
        unigramProb[alphabet] = unigramProb[alphabet] / totalCharacters
    
    # collect bigram counts
    for word in dataset:
        word = convertWord(word)
        # generate a list of bigrams
        bigram1 = [word[i] for i in range(len(word)-1)]
        bigram2 = [word[i+1] for i in range(len(word)-1)]
        bigramList = zip(bigram1, bigram2)
        # iterate over bigrams
        for bigram in bigramList:
            first, second = bigram
            bigramProb[first][second] += 1
    
    # compute probability for second alphabet given first alphabet
    for first in bigramProb:
        for second in bigramProb[first]:
            bigramProb[first][second] = bigramProb[first][second] / sum(bigramProb[first].values())

    # collect trigram counts
    for word in dataset:
        word = convertWord(word)
        # generate a list of trigrams
        trigram1 = [word[i] for i in range(len(word)-2)]
        trigram2 = [word[i+1] for i in range(len(word)-2)]
        trigram3 = [word[i+2] for i in range(len(word)-2)]
        trigramList = zip(trigram1, trigram2, trigram3)
        # iterate over trigrams
        for trigram in trigramList:
            first, second, third = trigram
            trigramProb[first][second][third] += 1
    
    # compute probability for third alphabet given first and second alphabets
    for first in trigramProb:
        for second in trigramProb[first]:
            bigramCount = sum(trigramProb[first][second].values())
            for third in trigramProb[first][second]:
                trigramProb[first][second][third] = trigramProb[first][second][third] / bigramCount

    # collect quadgram counts
    for word in dataset:
        word = convertWord(word)
        # generate a list of quadgrams
        quadgram1 = [word[i] for i in range(len(word)-3)]
        quadgram2 = [word[i+1] for i in range(len(word)-3)]
        quadgram3 = [word[i+2] for i in range(len(word)-3)]
        quadgram4 = [word[i+3] for i in range(len(word)-3)]
        quadgramList = zip(quadgram1, quadgram2, quadgram3, quadgram4)
        # iterate over trigrams
        for quadgram in quadgramList:
            first, second, third, fourth = quadgram
            quadgramProb[first][second][third][fourth] += 1
    
    # compute probability for fourth alphabet given first, second, and third alphabets
    for first in quadgramProb:
        for second in quadgramProb[first]:
            for third in quadgramProb[first][second]:
                trigramCount = sum(quadgramProb[first][second][third].values())
                for fourth in quadgramProb[first][second][third]:
                    quadgramProb[first][second][third][fourth] = quadgramProb[first][second][third][fourth] / trigramCount
        
    # collect pentgram counts
    for word in dataset:
        word = convertWord(word)
        # generate a list of quadgrams
        pentgram1 = [word[i] for i in range(len(word)-4)]
        pentgram2 = [word[i+1] for i in range(len(word)-4)]
        pentgram3 = [word[i+2] for i in range(len(word)-4)]
        pentgram4 = [word[i+3] for i in range(len(word)-4)]
        pentgram5 = [word[i+4] for i in range(len(word)-4)]
        pentgramList = zip(pentgram1, pentgram2, pentgram3, pentgram4, pentgram5)
        # iterate over trigrams
        for pentgram in pentgramList:
            first, second, third, fourth, fifth = pentgram
            pentgramProb[first][second][third][fourth][fifth] += 1
    
    # compute probability for fifth alphabet given first, second, third, and fourth alphabets
    for first in pentgramProb:
        for second in pentgramProb[first]:
            for third in pentgramProb[first][second]:
                for fourth in pentgramProb[first][second][third]:
                    quadgramCount = sum(pentgramProb[first][second][third][fourth].values())
                    for fifth in pentgramProb[first][second][third][fourth]:
                        pentgramProb[first][second][third][fourth][fifth] = pentgramProb[first][second][third][fourth][fifth] / quadgramCount

    return(unigramProb, bigramProb, trigramProb, quadgramProb, pentgramProb)

# get probabilities for unigram, bigram, trigram, quadgram, and pentgram
unigramProb, bigramProb, trigramProb, quadgramProb, pentgramProb = getNgramProb(trainSet)

def trigramGuesser(mask, guessed, **kwargs):
    # initialize variables
    uniProb = defaultdict(Counter)
    biProb = defaultdict(Counter) 
    triProb = defaultdict(Counter) 
    interpProb = defaultdict(Counter) 
    alphabets = set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
         'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
    alphabets = alphabets - guessed 
    mask = ["<s>","<s>"] + [char for char in mask]
    maskLength = len(mask)
    
    # get unigram, bigram, and trigram probabilities for each position
    for pos in range(2, maskLength):
        if (mask[pos] == '_'):
            for alphabet in alphabets:
                uniProb[pos][alphabet] = unigramProb[alphabet]
                biProb[pos][alphabet] = bigramProb[mask[pos-1]][alphabet]
                triProb[pos][alphabet] = trigramProb[mask[pos-2]][mask[pos-1]][alphabet]
    
    # parameters for linear interpolation
    lambdaUnigram = kwargs['lambdaUnigram']
    lambdaBigram = kwargs['lambdaBigram']
    lambdaTrigram = 1 - lambdaUnigram - lambdaBigram
    
    # sum all the vector of probabilities over each alphabet, to get a vector of scores for each alphabet     
    for alphabet in alphabets:
        interpProb[alphabet] = 0
        for pos in uniProb:
            interpProb[alphabet] += (lambdaUnigram * uniProb[pos][alphabet]) + (lambdaBigram * biProb[pos][alphabet]) + (lambdaTrigram * triProb[pos][alphabet])
    
    # return the alphabet which has the highest score
    return(max(interpProb, key=interpProb.get))    
        
print('Average number of mistakes for trigram guesser: ', hangmanPerformance(testSet, trigramGuesser, lambdaUnigram=0.1, lambdaBigram=0.0))  

def quadgramGuesser(mask, guessed, **kwargs):
    # initialize variables
    uniProb = defaultdict(Counter)
    biProb = defaultdict(Counter) 
    triProb = defaultdict(Counter) 
    quadProb = defaultdict(Counter) 
    interpProb = defaultdict(Counter) 
    alphabets = set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
         'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
    alphabets = alphabets - guessed 
    mask = ["<s>","<s>","<s>"] + [char for char in mask]
    maskLength = len(mask)
    
    # get unigram, bigram, trigram, and quadgram probabilities for each position
    for pos in range(3, maskLength):
        if (mask[pos] == '_'):
            for alphabet in alphabets:
                uniProb[pos][alphabet] = unigramProb[alphabet]
                biProb[pos][alphabet] = bigramProb[mask[pos-1]][alphabet]
                triProb[pos][alphabet] = trigramProb[mask[pos-2]][mask[pos-1]][alphabet]
                quadProb[pos][alphabet] = quadgramProb[mask[pos-3]][mask[pos-2]][mask[pos-1]][alphabet]

    # parameters for linear interpolation
    lambdaUnigram = kwargs['lambdaUnigram']
    lambdaBigram = kwargs['lambdaBigram']
    lambdaTrigram = kwargs['lambdaTrigram']
    lambdaQuadgram = 1 - lambdaUnigram - lambdaBigram - lambdaTrigram
    
    # sum all the vector of probabilities over each alphabet, to get a vector of scores for each alphabet   
    for alphabet in alphabets:
        interpProb[alphabet] = 0
        for pos in uniProb:
            interpProb[alphabet] += (lambdaUnigram * uniProb[pos][alphabet]) + (lambdaBigram * biProb[pos][alphabet]) + (lambdaTrigram * triProb[pos][alphabet]) + (lambdaQuadgram * quadProb[pos][alphabet])

    # return the alphabet which has the highest score
    return(max(interpProb, key=interpProb.get))    

print('Average number of mistakes for quadgram guesser: ', hangmanPerformance(testSet, quadgramGuesser, lambdaUnigram=0.1, lambdaBigram=0.0, lambdaTrigram=0.3))  

def pentgramGuesser(mask, guessed, **kwargs):
    # initialize variables
    uniProb = defaultdict(Counter)
    biProb = defaultdict(Counter) 
    triProb = defaultdict(Counter) 
    quadProb = defaultdict(Counter) 
    pentProb = defaultdict(Counter) 
    interpProb = defaultdict(Counter) 
    alphabets = set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
         'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
    alphabets = alphabets - guessed 
    mask = ["<s>","<s>","<s>","<s>"] + [char for char in mask]
    maskLength = len(mask)
    
    # get unigram, bigram, trigram, quadgram, and pentgram probabilities for each position
    for pos in range(4, maskLength):
        if (mask[pos] == '_'):
            for alphabet in alphabets:
                uniProb[pos][alphabet] = unigramProb[alphabet]
                biProb[pos][alphabet] = bigramProb[mask[pos-1]][alphabet]
                triProb[pos][alphabet] = trigramProb[mask[pos-2]][mask[pos-1]][alphabet]
                quadProb[pos][alphabet] = quadgramProb[mask[pos-3]][mask[pos-2]][mask[pos-1]][alphabet]
                pentProb[pos][alphabet] = pentgramProb[mask[pos-4]][mask[pos-3]][mask[pos-2]][mask[pos-1]][alphabet]

    # parameters for linear interpolation
    lambdaUnigram = kwargs['lambdaUnigram']
    lambdaBigram = kwargs['lambdaBigram']
    lambdaTrigram = kwargs['lambdaTrigram']
    lambdaQuadgram = kwargs['lambdaQuadgram']
    lambdaPentgram = 1 - lambdaUnigram - lambdaBigram - lambdaTrigram - lambdaQuadgram
    
    # sum all the vector of probabilities over each alphabet, to get a vector of scores for each alphabet       
    for alphabet in alphabets:
        interpProb[alphabet] = 0
        for pos in uniProb:
            interpProb[alphabet] += (lambdaUnigram * uniProb[pos][alphabet]) + (lambdaBigram * biProb[pos][alphabet]) + (lambdaTrigram * triProb[pos][alphabet]) + (lambdaQuadgram * quadProb[pos][alphabet]) + (lambdaPentgram * pentProb[pos][alphabet])

    # return the alphabet which has the highest score    
    return(max(interpProb, key=interpProb.get))    

print('Average number of mistakes for pentgram guesser: ', hangmanPerformance(testSet, pentgramGuesser, lambdaUnigram=0.1, lambdaBigram=0.0, lambdaTrigram=0.2, lambdaQuadgram=0.1))  

Average number of mistakes for trigram guesser:  8.341
Average number of mistakes for quadgram guesser:  7.837
Average number of mistakes for pentgram guesser:  7.649
